<a href="https://colab.research.google.com/github/AliFarouk22/Master/blob/master/Huma%20Detection%20Lightweight%20Deep%20Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Connect to Drive**

In [0]:
import tensorflow as tf
from keras import layers, models, losses, activations, optimizers, metrics, regularizers, initializers, callbacks

print(tf.VERSION)
print(tf.keras.__version__)


# from keras import layers, models, losses, activations, optimizers, metrics, regularizers, initializers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
# import tensorflow as tf

np.random.seed(123)
tf.set_random_seed(123)

from google.colab import drive
drive.mount('/content/drive')


# **GPU test**

In [0]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# **Imports**

In [0]:
import keras

from skimage import exposure
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import numpy as np
# import tensorflow as tf
import random as rn
from keras import backend as K

from keras import models
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform

from keras.applications import VGG16

from sklearn.model_selection import GridSearchCV
# from keras.models import Sequential
# from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# from keras.optimizers import SGD

import cv2
import os
import shutil

from keras.utils import plot_model

from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose, SeparableConv2D
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.optimizers import Adam, RMSprop, Nadam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import losses
from keras.activations import relu
from keras.layers import advanced_activations


# Code to make results ***reproducable***

In [0]:

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.

np.random.seed(42)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.

rn.seed(1234)

# Force TensorFlow to use single thread.
# Multiple threads are a potential source of non-reproducible results.
# For further details, see: https://stackoverflow.com/questions/42022950/

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)


# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see:
# https://www.tensorflow.org/api_docs/python/tf/set_random_seed

tf.set_random_seed(1234)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

# Rest of code follows ...

# ***Read full image 224 x 224***

In [0]:


# train_dir = '/content/drive/My Drive/Master/Dataset/INRIA/Train/train/' ## INRIA 
# train_dir = '/content/drive/My Drive/Master/Dataset/Stanford40_SomeClasses/' ## Stanford40 
# train_dir = '/content/drive/My Drive/Master/Dataset/ImageNet/train/' ## ImageNet 
train_dir = '/content/drive/My Drive/Master/Dataset/ImgNt_INR_Stan/' ## ALL datasets ImgNet_INR_Stan
# train_dir = '/content/drive/My Drive/Master/Dataset/ImgNt_INR_Sample/' ## ImgNt & INR Samples
validation_dir = '/content/drive/My Drive/Master/Dataset/INRIA/Train/validation/'


def contrast_stretching(img):
    p2, p98 = np.percentile(img, (2, 98))
    img_rescale = exposure.rescale_intensity(img, in_range=(p2, p98), out_range=(p2, p98))

    return img_rescale

datagen = ImageDataGenerator(rescale=1./255, 
                             preprocessing_function=contrast_stretching
)

batchTrain = 40
batchValidation = 40


train_generator = datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=batchTrain, class_mode='binary', shuffle=True, seed=1234)#, save_to_dir='G:\PyCharmWorkspace\Keras\\all\SavedFromKerasGenerator')
validation_generator = datagen.flow_from_directory(validation_dir, target_size=(224, 224), batch_size=batchValidation, class_mode='binary', seed=1234)
# test_generator = test_datagen.flow_from_directory(test_dir, target_size=(128, 64), batch_size=740, class_mode='binary')
# train_generator = train_datagen.flow(x=, batch_size=32, save_to_dir='')

print("Generator Sizes: ", train_generator.n, ", ", validation_generator.n)
print("Batch Sizes: ", train_generator.batch_size, ", ", validation_generator.batch_size)
print("Number of Classes: ", train_generator.num_classes, ", ", validation_generator.num_classes)
# print("classes: ", train_generator[0])
# print("classes: ", train_generator.shuffle)

print(train_generator[0][0].shape)
img = train_generator[0][0][10]
print("===================", np.max(img) , type(img[0][0][0]), img.mean())
plt.imshow(img)
plt.show()



# **Model**

In [0]:

dropout=0.35
mySeed = 1234
initializer = initializers.glorot_uniform(seed=mySeed)#truncated_normal(mean=0.0, stddev=0.5, seed=mySeed)#random_normal(mean=0.0, stddev=0.01, seed=mySeed)
l2Value = 0.001

model = models.Sequential()

model.add(layers.Conv2D(filters=64, kernel_size=(7, 7), #activation='relu',
                        input_shape=(224, 224, 3),
                        kernel_regularizer=regularizers.l2(l2Value),
                        strides=(1, 1), padding='same',
                        kernel_initializer=initializer
                       ))
model.add(layers.BatchNormalization(axis=3))
model.add(layers.Activation('relu'))
# model.add(layers.activations.relu())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(dropout, seed=mySeed))

#bottleneck
# model.add(layers.Conv2D(filters=20, kernel_size=(1, 1),
#                         kernel_regularizer=regularizers.l2(l2Value),
#                         strides=(1, 1), padding='same',
#                         kernel_initializer=initializer
#                         ))
# model.add(layers.Activation('relu'))

model.add(layers.SeparableConv2D(filters=64, kernel_size=(5, 5),# activation='relu',
                                 strides=(1, 1),
                                 padding='same', data_format='channels_last', depth_multiplier=1,
                                 depthwise_regularizer=regularizers.l2(l2Value),
                                 pointwise_regularizer=regularizers.l2(l2Value),
                                 depthwise_initializer=initializer, 
                                 pointwise_initializer=initializer
                                ))
model.add(layers.BatchNormalization(axis=3))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(dropout, seed=mySeed))

#bottleneck
# model.add(layers.Conv2D(filters=16, kernel_size=(1, 1),
#                         kernel_regularizer=regularizers.l2(l2Value),
#                         strides=(1, 1), padding='same',
#                         kernel_initializer=initializer
#                         ))
# model.add(layers.Activation('relu'))

model.add(layers.SeparableConv2D(filters=128, kernel_size=(3, 3),# activation='relu',
                                 strides=(1, 1),
                                 padding='same', data_format='channels_last', depth_multiplier=1,
                                 depthwise_regularizer=regularizers.l2(l2Value),
                                 pointwise_regularizer=regularizers.l2(l2Value),
                                 depthwise_initializer=initializer, 
                                 pointwise_initializer=initializer
                                ))
model.add(layers.BatchNormalization(axis=3))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(dropout, seed=mySeed))

#bottleneck
# model.add(layers.Conv2D(filters=16, kernel_size=(1, 1),
#                         kernel_regularizer=regularizers.l2(l2Value),
#                         strides=(1, 1), padding='same',
#                         kernel_initializer=initializer
#                         ))
# model.add(layers.Activation('relu'))

model.add(layers.SeparableConv2D(filters=128, kernel_size=(3, 3),# activation='relu',
                                 strides=(1, 1),
                                 padding='same', data_format='channels_last', depth_multiplier=1,
                                 depthwise_regularizer=regularizers.l2(l2Value),
                                 pointwise_regularizer=regularizers.l2(l2Value),
                                 depthwise_initializer=initializer, 
                                 pointwise_initializer=initializer
                                ))
model.add(layers.BatchNormalization(axis=3))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(dropout, seed=mySeed))

#bottleneck
# model.add(layers.Conv2D(filters=32, kernel_size=(1, 1),
#                         kernel_regularizer=regularizers.l2(l2Value),
#                         strides=(1, 1), padding='same',
#                         kernel_initializer=initializer
#                         ))
# model.add(layers.Activation('relu'))

model.add(layers.SeparableConv2D(filters=512, kernel_size=(3, 3),# activation='relu',
                                 strides=(1, 1),
                                 padding='same', data_format='channels_last', depth_multiplier=1,
                                 depthwise_regularizer=regularizers.l2(l2Value),
                                 pointwise_regularizer=regularizers.l2(l2Value),
                                 depthwise_initializer=initializer, 
                                 pointwise_initializer=initializer
                                ))
model.add(layers.BatchNormalization(axis=3))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(dropout, seed=mySeed))

#bottleneck
model.add(layers.Conv2D(filters=128, kernel_size=(1, 1),
                        kernel_regularizer=regularizers.l2(l2Value),
                        strides=(1, 1), padding='same',
                        kernel_initializer=initializer
                        ))
model.add(layers.Activation('relu'))

# 7*7 
model.add(layers.SeparableConv2D(filters=512, kernel_size=(7, 7),# activation='relu',
                                 strides=(1, 1),
                                 padding='valid', data_format='channels_last', depth_multiplier=1,
                                 depthwise_regularizer=regularizers.l2(l2Value),
                                 pointwise_regularizer=regularizers.l2(l2Value),
                                 depthwise_initializer=initializer, 
                                 pointwise_initializer=initializer
                                ))
model.add(layers.BatchNormalization(axis=3))
model.add(layers.Activation('relu'))

model.add(layers.Flatten())
# model.add(layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(l2Value) , kernel_initializer=initializer))
# model.add(layers.Dropout(dropout, seed=mySeed))
model.add(layers.Dense(1, activation='sigmoid', kernel_initializer=initializer))

print( model.summary() )


# **Model 2**

In [0]:

dropout=0.35
mySeed = 1234
initializer = initializers.glorot_uniform(seed=mySeed)#truncated_normal(mean=0.0, stddev=0.5, seed=mySeed)#random_normal(mean=0.0, stddev=0.01, seed=mySeed)
l2Value = 0.001

inputLayer = layers.Input(shape=(224, 224, 3))

layer = Conv2D(filters=64, kernel_size=(3, 3), #activation='relu',
                        kernel_regularizer=regularizers.l2(l2Value),
                        strides=(1, 1), padding='same',
                        kernel_initializer=initializer
                       ) (inputLayer)
layer = Activation('relu') (layer)
layer = Conv2D(filters=64, kernel_size=(3, 3), #activation='relu',
                        kernel_regularizer=regularizers.l2(l2Value),
                        strides=(1, 1), padding='same',
                        kernel_initializer=initializer
                       ) (layer)
layer = Activation('relu') (layer)
layer = Conv2D(filters=64, kernel_size=(3, 3), #activation='relu',
                        kernel_regularizer=regularizers.l2(l2Value),
                        strides=(1, 1), padding='same',
                        kernel_initializer=initializer
                       ) (layer)
layer = BatchNormalization() (layer)
layer = Activation('relu') (layer)
layer = MaxPooling2D((2, 2)) (layer)
layer = Dropout(dropout, seed=mySeed) (layer)

layer = SeparableConv2D(filters=64, kernel_size=(3, 3),# activation='relu',
                                 strides=(1, 1),
                                 padding='same', data_format='channels_last', depth_multiplier=1,
                                 depthwise_regularizer=regularizers.l2(l2Value),
                                 pointwise_regularizer=regularizers.l2(l2Value),
                                 depthwise_initializer=initializer, 
                                 pointwise_initializer=initializer
                                ) (layer)
layer = Activation('relu') (layer)
layer = SeparableConv2D(filters=64, kernel_size=(3, 3),# activation='relu',
                                 strides=(1, 1),
                                 padding='same', data_format='channels_last', depth_multiplier=1,
                                 depthwise_regularizer=regularizers.l2(l2Value),
                                 pointwise_regularizer=regularizers.l2(l2Value),
                                 depthwise_initializer=initializer, 
                                 pointwise_initializer=initializer
                                ) (layer)
layer = BatchNormalization() (layer)
layer = Activation('relu') (layer)
layer = MaxPooling2D((2, 2)) (layer)
layer = Dropout(dropout, seed=mySeed) (layer)

layer = SeparableConv2D(filters=128, kernel_size=(3, 3),# activation='relu',
                                 strides=(1, 1),
                                 padding='same', data_format='channels_last', depth_multiplier=1,
                                 depthwise_regularizer=regularizers.l2(l2Value),
                                 pointwise_regularizer=regularizers.l2(l2Value),
                                 depthwise_initializer=initializer, 
                                 pointwise_initializer=initializer
                                ) (layer)
layer = BatchNormalization() (layer)
layer = Activation('relu') (layer)
layer = MaxPooling2D((2, 2)) (layer)
layer = Dropout(dropout, seed=mySeed) (layer)

layer = SeparableConv2D(filters=128, kernel_size=(3, 3),# activation='relu',
                                 strides=(1, 1),
                                 padding='same', data_format='channels_last', depth_multiplier=1,
                                 depthwise_regularizer=regularizers.l2(l2Value),
                                 pointwise_regularizer=regularizers.l2(l2Value),
                                 depthwise_initializer=initializer, 
                                 pointwise_initializer=initializer
                                ) (layer)
layer = BatchNormalization() (layer)
layer = Activation('relu') (layer)
layer = MaxPooling2D((2, 2)) (layer)
layer = Dropout(dropout, seed=mySeed) (layer)

layer = SeparableConv2D(filters=512, kernel_size=(3, 3),# activation='relu',
                                 strides=(1, 1),
                                 padding='same', data_format='channels_last', depth_multiplier=1,
                                 depthwise_regularizer=regularizers.l2(l2Value),
                                 pointwise_regularizer=regularizers.l2(l2Value),
                                 depthwise_initializer=initializer, 
                                 pointwise_initializer=initializer
                                ) (layer)
layer = BatchNormalization() (layer)
layer = Activation('relu') (layer)
layer = MaxPooling2D((2, 2)) (layer)
layer = Dropout(dropout, seed=mySeed) (layer)

#bottleneck
layer = Conv2D(filters=128, kernel_size=(1, 1),
                        kernel_regularizer=regularizers.l2(l2Value),
                        strides=(1, 1), padding='same',
                        kernel_initializer=initializer
                        ) (layer)
layer = Activation('relu') (layer)

# 7*7 
layer = SeparableConv2D(filters=512, kernel_size=(7, 7),# activation='relu',
                                 strides=(1, 1),
                                 padding='valid', data_format='channels_last', depth_multiplier=1,
                                 depthwise_regularizer=regularizers.l2(l2Value),
                                 pointwise_regularizer=regularizers.l2(l2Value),
                                 depthwise_initializer=initializer, 
                                 pointwise_initializer=initializer
                                ) (layer)
layer = BatchNormalization() (layer)
layer = Activation('relu') (layer)

layer = layers.Flatten() (layer)
outputs = Dense(1, activation='sigmoid', kernel_initializer=initializer) (layer)

model = Model(inputs=[inputLayer], outputs=[outputs])

print( model.summary() )


# **Compile Model**

In [0]:
model.compile(optimizer=optimizers.Adam(lr=0.0007),# decay=0.001),
              loss=losses.binary_crossentropy,
              metrics=[metrics.binary_accuracy]
             )


print( model.summary() )


# **VGG layers weights to model**

In [0]:


conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))

conv_base.summary()

model.summary()

print('My Model weights: ', type(model.get_weights()), len(model.get_weights()))
print('VGG16 Model weights: ', type(conv_base.get_weights()), len(conv_base.get_weights()))

vggWeights = np.empty(0)

for vggLayer in conv_base.layers:

  if "conv" in vggLayer.name:
  
    vggModelLayerWeightsFlattened = vggLayer.get_weights()[0].flatten()
    print("Flattened VGG Weights Shape: ", vggModelLayerWeightsFlattened.shape, ", ", vggLayer.get_weights()[0].shape)
    vggWeights = np.concatenate((vggWeights, vggModelLayerWeightsFlattened), axis=None)
    
print("VGG weights flattened appended Length: ", vggWeights.shape)

i = 0

for lay in model.layers:
  
  print(lay.name)
  lastStop = 0
  
  if "conv" in lay.name:
    
    vggWeightsToModelWeights = []
    numWeightsArraysInLayer = len(lay.get_weights())
    print("numWeightsArraysInLayer: ", numWeightsArraysInLayer)
    
    for j in range(numWeightsArraysInLayer):
      layerSize = lay.get_weights()[j].shape
      print("layerSize: ", layerSize)
      layerShapeMultiplied = 1
      
      for dim in layerSize:
        layerShapeMultiplied *= dim
      
      x = vggWeights[lastStop: (lastStop + layerShapeMultiplied)].reshape(layerSize)
      lastStop = lastStop + layerShapeMultiplied
      vggWeightsToModelWeights.append(x)
      
    print ("Sizes of vggWeightsToModelWeights: ", len(vggWeightsToModelWeights))
    for weights in vggWeightsToModelWeights:
      print(len(weights))
      
    model.layers[i].set_weights(vggWeightsToModelWeights)
    
  i += 1




# **Run Model**

In [0]:
print( model.summary() )
callbacks = [
    # EarlyStopping(patience=3, verbose=1),
    ReduceLROnPlateau(factor=0.9, patience=10, min_lr=0.0001, verbose=1),
    # ReduceLROnPlateau(factor=0.01, patience=3, min_lr=0.0001, verbose=1),
    ModelCheckpoint("/content/drive/My Drive/Master/INRIA_images_hog/MyIdea/my_keras_model_bottleneck_103_1.h5",
                                          save_best_only=True)#, save_weights_only=True)
]

history = model.fit_generator(train_generator,
                              steps_per_epoch=train_generator.n/batchTrain, #((train_generator.n * transformationNumber)/batch),
                              epochs=100,
                              validation_data=validation_generator,
                              validation_steps=(validation_generator.n/batchValidation), 
                              callbacks=callbacks)


acc = history.history['binary_accuracy']
val_acc = history.history['val_binary_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()



# Test

In [0]:

model = models.load_model('/content/drive/My Drive/Master/INRIA_images_hog/MyIdea/my_keras_model_bottleneck_103_1.h5')#my_keras_model_bottleneck_9.h5')

plot_model(model, to_file='/content/drive/My Drive/Master/INRIA_images_hog/CNN_Models/model.png')

# test_dir = '/content/drive/My Drive/Master/Dataset/INRIA/Test/test/'
test_dir_pos = '/content/drive/My Drive/Master/Dataset/INRIA/Test/test/pos/' #AllData/test/pos/'
test_dir_neg = '/content/drive/My Drive/Master/Dataset/INRIA/Test/test/neg/' #AllData/test/neg/'
# test_dir_pos = '/content/drive/My Drive/Master/Dataset/GeneratedTest/allImages/'#workingPos#allImages/' #AllData/test/pos/'
# test_dir_pos = '/content/drive/My Drive/Master/Dataset/GeneratedTest/Ali_and_Ahmed/'


test_generator_pos = datagen.flow_from_directory(test_dir_pos, target_size=(224, 224), batch_size=1, class_mode='binary', seed=1234)
print("classes: ", test_generator_pos.classes)
test_generator_neg = datagen.flow_from_directory(test_dir_neg, target_size=(224, 224), batch_size=1, class_mode='binary', seed=1234)
print("classes: ", test_generator_neg.classes)

pos11 = np.round( model.predict_generator(test_generator_pos, steps=test_generator_pos.n) )
pos1 = np.mean( np.round( model.predict_generator(test_generator_pos, steps=test_generator_pos.n) ) )
neg1 = 1 - np.mean( np.round( model.predict_generator(test_generator_neg, steps=test_generator_neg.n) ) )

pos11 = np.reshape(pos11, newshape=(1, -1))
print("__________________________________ SHAPE: ", pos11.shape)
names = np.asarray(test_generator_pos.filenames)
for index in range(pos11.shape[1]):
  if pos11[0, index] == 1.0:
    imageName = names[index]
    print ("index: ", index, "     prediction: ", pos11[0, index], "     imageName: ", imageName)
    # imagePath = test_dir_pos + imageName
    # shutil.copy((imagePath) , ("/content/drive/My Drive/Master/Dataset/GeneratedTest/Ok3/" + imageName))
#     # copy( (imagePath) , ("/content/drive/My Drive/Master/Dataset/GeneratedTest/Ok/" + imageName) )Ali_and_Ahmed2




print( "pos: ", pos11, "\n", np.sum(pos11) ) 
print( "pos: ", pos1 ) 
print( "neg: ", neg1 )

print( "True pos: ", pos1 ) 
print( "True neg: ", neg1 )
print( "False pos:", 1 - neg1 )
print( "False neg:", 1 - pos1 )

# Accuracy = (TP+TN)/(TP+TN+FP+FN)
# •	F-score = 2TP / (2TP + FP + FN)

totalAccuracy = (pos1 + neg1) / (pos1 + neg1 + (1-neg1) + (1-neg1))#(pos * (test_generator_pos.n/(test_generator_pos.n + test_generator_neg.n)) ) + (neg * (test_generator_neg.n/(test_generator_pos.n + test_generator_neg.n)) )
F_Score = (2*pos1) / ( (2*pos1) + (1-neg1) + (1-pos1) )
print("Total Accuracy: ", totalAccuracy, "F-Score", F_Score)

################################################


print ("#####-------------------- Test on ImageNet -------------------#####")
test_dir_pos_ImageNet = '/content/drive/My Drive/Master/Dataset/ImageNet/test/pos/' #AllData/test/pos/'
test_dir_neg_ImageNet = '/content/drive/My Drive/Master/Dataset/ImageNet/test/neg/' #AllData/test/neg/'

test_generator_pos = datagen.flow_from_directory(test_dir_pos_ImageNet, target_size=(224, 224), batch_size=1, class_mode='binary')
print("classes: ", test_generator_pos.classes)
test_generator_neg = datagen.flow_from_directory(test_dir_neg_ImageNet, target_size=(224, 224), batch_size=1, class_mode='binary')
print("classes: ", test_generator_neg.classes)


pos2 = np.mean( np.round( model.predict_generator(test_generator_pos, steps=test_generator_pos.n) ) )
neg2 = 1 - np.mean( np.round( model.predict_generator(test_generator_neg, steps=test_generator_neg.n) ) )


print( "pos: ", pos2 ) 
print( "neg: ", neg2 )

print( "True pos: ", pos2 ) 
print( "True neg: ", neg2 )
print( "False pos:", 1 - neg2 )
print( "False neg:", 1 - pos2 )


totalAccuracy = (pos2 + neg2) / (pos2 + neg2 + (1-neg2) + (1-neg2))#(pos * (test_generator_pos.n/(test_generator_pos.n + test_generator_neg.n)) ) + (neg * (test_generator_neg.n/(test_generator_pos.n + test_generator_neg.n)) )
F_Score = (2*pos2) / ( (2*pos2) + (1-neg2) + (1-pos2) )
print("Total Accuracy: ", totalAccuracy, "F-Score", F_Score)

# ##################################

print ("#####-------------------- Test on Pascal Voc 2007 -------------------#####")
test_dir_pos_ImageNet = '/content/drive/My Drive/Master/Dataset/PascalVoc2007/test/pos/' #AllData/test/pos/'

test_generator_pos = datagen.flow_from_directory(test_dir_pos_ImageNet, target_size=(224, 224), batch_size=1, class_mode='binary')
print("classes: ", test_generator_pos.classes)


pos3 = np.mean( np.round( model.predict_generator(test_generator_pos, steps=test_generator_pos.n) ) )


print( "pos: ", pos3 ) 

print( "True pos: ", pos3 ) 
print( "False neg:", 1 - pos3 )

# F-Score = (2*pos) / ( (2*pos) + (1-neg) + (1-pos))

print ("#####-------------------- Test on All Datasets -------------------#####")

totalPos = (pos1 + pos2 + pos3) / 3
totalNeg = (neg1 + neg2) / 2


print( "total pos: ", totalPos ) 
print( "total neg: ", totalNeg )

print( "True pos: ", totalPos ) 
print( "True neg: ", totalNeg )
print( "False pos:", 1 - totalNeg )
print( "False neg:", 1 - totalPos )


totalAccuracy = (totalPos + totalNeg) / (totalPos + totalNeg + (1-totalNeg) + (1-totalNeg))#(pos * (test_generator_pos.n/(test_generator_pos.n + test_generator_neg.n)) ) + (neg * (test_generator_neg.n/(test_generator_pos.n + test_generator_neg.n)) )
F_Score = (2*totalPos) / ( (2*totalPos) + (1-totalNeg) + (1-totalPos) )
print("Total Accuracy: ", totalAccuracy, "F-Score", F_Score)


# **------------------------------------**